In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, parser='auto')
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist["data"], mnist["target"]

In [4]:
y = y.astype(np.uint8)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [6]:
print(y.shape)
print(np.array(y).reshape(-1,1).shape)
y.unique()
# 아래 구문에서 오류 발생: Y = enc.transform(y[:,np.newaxis]).toarray()
# 오류: ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.
# 오류 해결을 위해 np.array로 변환 후  수정 (이후 1곳도 똑같이 수정)

(70000,)
(70000, 1)


array([5, 0, 4, 1, 9, 2, 3, 6, 7, 8], dtype=uint8)

In [7]:
enc.fit(np.array(y).reshape(-1,1))

OneHotEncoder()

In [8]:
# one-hot encoder 정보
print("category count:",enc.n_features_in_)
print(enc.categories_)

category count: 1
[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)]


In [9]:
Y = enc.transform(np.array(y).reshape(-1,1)).toarray()

In [10]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], Y[:60000], Y[60000:]

In [11]:
X_train = X_train / 255
X_test = X_test / 255

In [12]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [13]:
def softmax(X, W):
    K = np.size(W, 1)
    A = np.exp(X @ W)
    B = np.diag(1 / (np.reshape(A @ np.ones((K,1)), -1)))
    Y = B @ A
    return Y

In [14]:
def compute_cost_cross_entropy(X, T, W):
    epsilon = 1e-5
    N = len(T)
    K = np.size(T, 1)
    cost = - (1/N) * np.ones((1,N)) @ (np.multiply(np.log(softmax(X, W) + epsilon), T)) @ np.ones((K,1))
    return cost

def compute_cost_L2(X, T, W, regularization_parameter):
    
    l2_value = np.sum(np.power(W,2))
    N = len(T)
    return compute_cost_cross_entropy(X, T, W) + l2_value * regularization_parameter

    

In [15]:
def predict(X, W):
    return np.argmax((X @ W), axis=1)

In [16]:
PRINT_COST_HISTORY=True

def batch_gd(X, T, W, learning_rate, iterations, batch_size, regularization_parameter):
    N = len(T)
    cost_history = np.zeros((iterations,1))
    shuffled_indices = np.random.permutation(N)
    X_shuffled = X[shuffled_indices]
    T_shuffled = T[shuffled_indices]

    for i in range(iterations):
        j = i % N
        X_batch = X_shuffled[j:j+batch_size]
        T_batch = T_shuffled[j:j+batch_size]
        # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌
        if X_batch.shape[0] < batch_size:
            X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))
            T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))
        W = W - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch))
        cost_history[i] = compute_cost_L2(X_batch, T_batch, W, regularization_parameter)
        if PRINT_COST_HISTORY & (i % 1000 == 0):
            print(cost_history[i][0])
            pass

    return (cost_history, W)

In [17]:
X = np.hstack((np.ones((np.size(X_train, 0),1)),X_train))
T = y_train

K = np.size(T, 1)
M = np.size(X, 1)
W = np.zeros((M,K))

iterations = 50000
learning_rate = 0.01

regularization_parameter = 0

initial_cost = compute_cost_L2(X, T, W, regularization_parameter)
ic_print = initial_cost[0][0]
print(f"regularization_parameter:{regularization_parameter}")
print("Initial Cost is: {} \n".format(initial_cost[0][0]))

(cost_history, W_optimal) = batch_gd(X, T, W, learning_rate, iterations, 64, regularization_parameter)

regularization_parameter:0
Initial Cost is: 2.3024850979937166 

2.2856295547388736
0.533706773861257
0.42612309186042546
0.27603846097365725
0.33006763959687463
0.34601757232745806
0.25667485469011564
0.3537483399469502
0.2575869655113071
0.30443660334905054
0.3792868011507706
0.41865959757732735
0.2890150931062271
0.30405594962982024
0.16882719256125267
0.2172255465070284
0.41178749256657043
0.41279901147785764
0.3939088123527864
0.29360361873789936
0.28935427898102845
0.16374274909996844
0.222191771145731
0.3852845141285202
0.37662850388582303
0.30044861495818037
0.38435512240742525
0.33468956718872134
0.30664792151942455
0.10237409202305728
0.5133677127815894
0.30801633811027546
0.18680118195842635
0.18329545196139843
0.34584578453250914
0.2501962019146978
0.2560984015799574
0.30987624028885413
0.27591304545895234
0.22359983250985405
0.2034900462895348
0.33435170331719305
0.37037946053111
0.18114823542827785
0.13755521487032782
0.11072901526372739
0.31530410747555326
0.309159006536

In [18]:
## Accuracy
X_ = np.hstack((np.ones((np.size(X_test, 0),1)),X_test))
T_ = y_test
y_pred = predict(X_, W_optimal)
score = float(sum(y_pred == np.argmax(T_, axis=1)))/ float(len(y_test))

print(score)

0.918


In [1]:
#optimize regularization_parameter, with multithreading

PRINT_COST_HISTORY = False

import multiprocessing.dummy as mp
from multiprocessing.dummy import Pool
import threading



def test_work(regularization_parameter, learning_rate):
    PRINT_COST_HISTORY = False
    X = np.hstack((np.ones((np.size(X_train, 0),1)),X_train))
    T = y_train

    K = np.size(T, 1)
    M = np.size(X, 1)
    W = np.zeros((M,K))

    iterations = 50000

    #initial_cost = compute_cost_L2(X, T, W, regularization_parameter)
    #ic_print = initial_cost[0][0]
    #print(f"regularization_parameter:{regularization_parameter}")
    #print("Initial Cost is: {} \n".format(initial_cost[0][0]))
    #print(f"start {regularization_parameter:.2f}")
    (cost_history, W_optimal) = batch_gd(X, T, W, learning_rate, iterations, 64, regularization_parameter)
    ## Accuracy
    X_ = np.hstack((np.ones((np.size(X_test, 0),1)),X_test))
    T_ = y_test
    y_pred = predict(X_, W_optimal)
    score = float(sum(y_pred == np.argmax(T_, axis=1)))/ float(len(y_test))
    return_str = f"regularization_parameter:{regularization_parameter:.4f},score:{score}"
    
    return (cost_history, W_optimal, score, return_str)





In [20]:
'''
hlist=list()
for i in range(0,100):
    hlist.append(list())
    learning_rate = 0.001 + i*i* 0.001
    for j in range(0,100): 
        regs = j * 0.2
        result = test_work(regs)
        pool.close()
        pool.join()
        print(f"learning_rate:{learning_rate:.4f},{result[3]}")
        result_arg = [learning_rate, regs, result[2],cost_history]
        hlist[-1].append(result_arg)
'''
    


              
              

'\nwith Pool(2) as p:\n    regs = [i * 0.2 for i in range(0,100)]\n    result = p.map(test_work, regs)\n    p.close()\n    p.join()\n'